# Generate Synthetic Test Set

## Set Up 

In [6]:
%pip install -q -U python-dotenv ipywidgets nest-asyncio langchain langchain-community langchain-chroma langchain-openai ragas 


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Load .env variables into the environment
from pathlib import Path
from dotenv import load_dotenv  
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

False

## Load Documents

In [2]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.text import TextLoader 

loader = DirectoryLoader("../../knowledge/originals/", glob="**/*.txt", loader_cls=TextLoader, show_progress=True) 
documents = loader.load() 

100%|██████████| 8/8 [00:00<00:00, 2300.14it/s]


In [3]:
for document in documents:
    document.metadata['filename'] = document.metadata['source']

# Generate Test Set

In [7]:
# Needed for async calls in Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.embeddings.base import LangchainEmbeddingsWrapper
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    LangchainEmbeddingsWrapper(embeddings)
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [13]:
# Convert testset to pandas dataframe and save to csv
data = testset.to_pandas()
data.to_csv("testset.csv", index=False)

In [21]:
# Generate HF dataset from pandas dataframe
from datasets import Dataset
hf_testset = Dataset.from_pandas(data)

In [ ]:
# Save HF dataset to disk
hf_testset.save_to_disk("hf_testset")

In [23]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate

In [ ]:
# results = evaluate(hf_testset)